# Serie 3
## Esercizio 1

In [3]:
class Worker implements Runnable {
        public static boolean isRunning = false;
        public static int finished = 0;

        private int count = 0;
        private final int id;
        private final Random random;

        public Worker(final int id) {
                this.id = id;
                this.random = new Random();
        }

        @Override
        public void run() {
                System.out.println("Worker" + id + " waiting to start");
                while (!isRunning) {
                        // Wait!
                }

                System.out.println("Worker" + id + " started");
                for (int i = 0; i < 10; i++) {
                        count += random.nextInt(40) + 10;
                        try {
                                Thread.sleep(random.nextInt(151) + 100);
                        } catch (final InterruptedException e) {
                                e.printStackTrace();
                        }
                }

                System.out.println("Worker" + id + " finished");
                finished++;
        }

        public void printResult() {
                System.out.println("Worker" + id + " reached " + count);
        }
}


final List <Worker> allWorkers = new ArrayList < > ();
final List <Thread> allThread = new ArrayList < > ();
for (int i = 1; i <= 10; i++) {
    final Worker target = new Worker(i);
    allWorkers.add(target);
    final Thread e = new Thread(target);
    allThread.add(e);
    e.start();
}

try {
    Thread.sleep(1000);
} catch (final InterruptedException e){

}

System.out.println("Main thread starting the race!");
Worker.isRunning = true;

Thread t = new Thread(() -> {
    try {
        Thread.sleep(5000);
        System.out.println("Timed out.");
        System.exit(1);
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
});

t.start();

while (Worker.finished < allWorkers.size()){
    // Wait
    Thread.sleep(50);
}

for (Worker worker: allWorkers){
    worker.printResult();
}

for (final Thread thread: allThread){
    try {
        thread.join();
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}

Worker6 waiting to start
Worker1 waiting to start
Worker3 waiting to start
Worker2 waiting to start
Worker5 waiting to start
Worker7 waiting to start
Worker8 waiting to start
Worker4 waiting to start
Worker9 waiting to start
Worker10 waiting to start
Main thread starting the race!
Timed out.


IllegalStateException: JShell (jdk.jshell.JShell@39c78df5) has been closed.

### Problematica
Il thread principale va in uno stato di hang perché Worker.finished viene messo in una cache diversa, a seconda del thread. Di conseguenza la modifica non è propagata agli altri thread ed i worker vanno in stato di hang.

### Risoluzione

Per ovviare al problema dobbiamo rendere la variabile statica "isRunning" di Worker `volatile`, in modo che questa sia sempre aggiornata in tutti i thread e che quindi non causi
dei cicli while infiniti causati dall'espressione `!isRunning`.

In [2]:
import java.util.concurrent.locks.Lock;
import java.util.concurrent.locks.ReentrantLock;
class Worker implements Runnable {
    public volatile static boolean isRunning = false;
    public volatile static int finished = 0;
    public static Lock lock = new ReentrantLock();
    private final int id;
    private final Random random;
    private int count = 0;

    public Worker(final int id) {
        this.id = id;
        this.random = new Random();
    }

    public static int getFinishedWorkers(){
        return finished;
    }

    @Override
    public void run() {
        System.out.println("Worker" + id + " waiting to start");
        while (!isRunning) {
            // Wait!
        }

        System.out.println("Worker" + id + " started");
        for (int i = 0; i < 10; i++) {
            count += random.nextInt(40) + 10;
            try {
                Thread.sleep(random.nextInt(151) + 100);
            } catch (final InterruptedException e) {
                e.printStackTrace();
            }
        }

        System.out.println("Worker" + id + " finished");
        Worker.lock.lock();
        finished++;
        Worker.lock.unlock();
    }

    public void printResult() {
        System.out.println("Worker" + id + " reached " + count);
    }
}

final List <Worker> allWorkers = new ArrayList < > ();
final List <Thread> allThread = new ArrayList < > ();
for (int i = 1; i <= 10; i++) {
    final Worker target = new Worker(i);
    allWorkers.add(target);
    final Thread e = new Thread(target);
    allThread.add(e);
    e.start();
}

try {
    Thread.sleep(1000);
} catch (final InterruptedException e){

}

System.out.println("Main thread starting the race!");
Worker.isRunning = true;

Thread t = new Thread(() -> {
    try {
        Thread.sleep(10000);
        System.out.println("Timed out.");
        System.exit(1);
    } catch (InterruptedException e) {
    }
});

t.start();


while (Worker.getFinishedWorkers() < allWorkers.size()){
    // Wait
}

t.interrupt();

for (Worker worker: allWorkers){
    worker.printResult();
}

for (final Thread thread: allThread){
    try {
        thread.join();
    } catch (final InterruptedException e) {
        e.printStackTrace();
    }
}

Worker6 waiting to start
Worker8 waiting to start
Worker7 waiting to start
Worker9 waiting to start
Worker4 waiting to start
Worker5 waiting to start
Worker2 waiting to start
Worker3 waiting to start
Worker1 waiting to start
Worker10 waiting to start
Main thread starting the race!
Worker2 started
Worker1 started
Worker7 started
Worker5 started
Worker10 started
Worker3 started
Worker8 started
Worker9 started
Worker4 started
Worker6 started
Worker4 finished
Worker5 finished
Worker2 finished
Worker10 finished
Worker8 finished
Worker7 finished
Worker1 finished
Worker9 finished
Worker6 finished
Worker3 finished
Worker1 reached 230
Worker2 reached 336
Worker3 reached 318
Worker4 reached 310
Worker5 reached 260
Worker6 reached 333
Worker7 reached 257
Worker8 reached 278
Worker9 reached 291
Worker10 reached 377


Vediamo quindi che, una volta corretto il codice inserendo `volatile`, il tutto funziona nel modo corretto.

## Esercizio 2


### Prima versione
La presente versione non utilizza nessuna opzione di sincronizzazione, di conseguenza i thread vanno in uno stado di hang ed il thread principale non termina mai.

### Seconda versione
Utilizzando la keyword `volatile` su `Counter.value` otteniamo un risultato 